In [1]:
from keras import datasets
from keras.utils import to_categorical
import numpy as np
np.random.seed(0)

(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

print((X_train.shape, y_train.shape), (X_test.shape, y_test.shape) )

Using CNTK backend


GPU
((60000, 28, 28), (60000,)) ((10000, 28, 28), (10000,))


In [2]:
# image scale (60000, 28, 28, 1)
# CNN
X_train =  (X_train.astype(np.float32)/255.0).reshape(X_train.shape[0], 28, 28, 1)
X_test =  (X_test.astype(np.float32)/255.0).reshape(X_test.shape[0], 28, 28, 1)

# MLP
# X_train =  (X_train.astype(np.float32)/255.0).reshape(X_train.shape[0], 784)
# X_test =  (X_test.astype(np.float32)/255.0).reshape(X_test.shape[0], 784)


In [3]:
# one-hot encoding
y_train_oh = to_categorical(y_train, num_classes=10, dtype='float32')
y_test_oh = to_categorical(y_test, num_classes=10, dtype='float32')

In [4]:
# # build model MLP
# from keras.models import Sequential
# from keras.layers import Dense, Conv2D, Flatten , MaxPooling2D
# # 3,741,066
# model = Sequential()
# model.add(Dense(4096, input_shape=(1, 784), activation='relu' ) )
# model.add(Dense(128, activation='relu'))
# model.add(Dense(10, activation='softmax'))
# print(model.summary())

In [5]:
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
#MaxPooling2D?

In [8]:
# build model 
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
model = Sequential()

model.add(Conv2D(filters=8, kernel_size=(5, 5), strides=(1, 1), input_shape=(28, 28, 1), 
                 padding='same', activation='relu' ) ) # (5*5 + 1) * 8 = 208
model.add(MaxPooling2D(pool_size=(5, 5), strides=(2, 2)))
model.add(Conv2D(filters=16, kernel_size=(5, 5), strides=(1, 1),
                 padding='same', activation='relu' ) ) # 8(previous filters)*(5*5) + 1(bias) = 201 , 201 * 16(current) = 3216
model.add(MaxPooling2D(pool_size=(5, 5), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax')) # 784 * 10 + 10 = 7850
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 28, 28, 8)         208       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 12, 12, 8)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 12, 12, 16)        3216      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 500)               128500    
_________________________________________________________________
dropout_1 (Dropout)          (None, 500)               0         
__________

In [ ]:
# compile build
import keras
from keras import optimizers
model.compile(optimizer=optimizers.SGD(0.2)  ,
              loss=keras.losses.categorical_crossentropy , 
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train_oh, batch_size=64, epochs= 10, verbose=1, validation_split=0.2)

In [ ]:
#model.evaluate?

In [ ]:
model.evaluate(X_train, y_train_oh, batch_size=512)[1], model.evaluate(X_test, y_test_oh, batch_size=512)[1]
#(0.9876000000635783, 0.9844999995231628)

In [ ]:
(1- model.evaluate(X_train, y_train_oh, batch_size=512)[1])*100, (1- model.evaluate(X_test, y_test_oh, batch_size=512)[1]) * 100
# (1.2399999936421713, 1.5500000476837195)
# (0.768333326975501, 1.0800000286102285)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(range(len(history.epoch)), history.history['acc'], color='red'  )
plt.plot(range(len(history.epoch)), history.history['val_acc'], color='blue'  )
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.grid()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(len(history.epoch)), history.history['loss'], color='red'  )
plt.plot(range(len(history.epoch)), history.history['val_loss'], color='blue'  )
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid()
plt.show()